## Сбор данных

In [1]:
import pandas as pd
import numpy as np
import requests

Работа посвещана созданию рекомендательной системы фильмов для онлайн кинотеатров. Для этого потребуются соответствующие данные о фильмах и их рейтингах.

Загружаем мета-данные фильмов [с kaggle](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset/data). Набор данных состоит из более 45000 фильмов, выпущенных до июля 2017 года.

#### Содержание файлов:

- <b>Movies_metadata.csv</b>: основной файл метаданных фильмов. Содержит информацию о 45 000 фильмах, представленных в наборе данных Full MovieLens. Характеристики включают posters, backdrops, budget, revenue, release dates, languages, production countries и companies.
- <b>keywords.csv</b>: содержит ключевые слова сюжета (movie plot keywords) для наших фильмов MovieLens. Доступен в виде строкового объекта JSON.
- <b>credits.csv</b>: Содержит информацию об актерах и съемочной группе (Cast and Crew) всех наших фильмов. Доступен в виде строкового объекта JSON.
- <b>links.csv</b>: файл, содержащий TMDB и идентификаторы IMDB всех фильмов, представленных в наборе данных Full MovieLens.
- <b>link_small.csv</b>: содержит TMDB и идентификаторы IMDB небольшого подмножества из 9000 фильмов полного набора данных.
- <b>ratings_small.csv</b>: Подмножество из 100 000 оценок 700 пользователей к 9 000 фильмам.

In [2]:
old_keywords = pd.read_csv('data/keywords.csv')
old_links = pd.read_csv('data/links.csv')
old_retings = pd.read_csv('data/ratings.csv')
old_credits = pd.read_csv("data/credits.csv")
movies_metadata = pd.read_csv("data/movies_metadata.csv")

movies_metadata.head(3)

C:\Users\Valeriia\AppData\Local\Temp\ipykernel_84388\1521140142.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv("data/movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


Данные достаточно старые, поэтому я решила скачать полный набор [MovieLens](https://grouplens.org/datasets/movielens/latest/)

Согласно приложенному файлу описания датасета, он был создан 20 июля 2023 г., поэтому должен содержать информацию о новых фильмах, выпущенных к этому времени.


#### Содержание файлов: 
 
- <b>«genome-tags.csv»</b> и <b>«genome-scores.csv»</b> - геном тега. Геном тега кодирует, насколько сильно фильмы демонстрируют определенные свойства, представленные тегами. Геном тега был рассчитан с использованием алгоритма машинного обучения на основе пользовательского контента, включая теги, рейтинги и текстовые обзоры. Геном разделен на два файла. Файл <b>genome-scores.csv</b> содержит данные о релевантности тегов фильмов в следующем формате:  movieId,tagId,relevance. Второй файл, <b>genome-tags.csv</b>, содержит описания тегов для идентификаторов тегов в файле генома в следующем формате: tagId,tag. 
- <b>«links.csv»</b> - идентификаторы, которые можно использовать для ссылки на другие источники данных фильма. Каждая строка этого файла после строки заголовка представляет один фильм и имеет следующий формат: movieId, imdbId, tmdbId
- <b>«movies.csv»</b> - фильмы. Каждая строка этого файла после строки заголовка представляет один фильм и имеет следующий формат: movieId, title, genres 
- <b>«ratings.csv»</b> - рейтинги. Каждая строка этого файла после строки заголовка представляет одну оценку одного фильма одним пользователем и имеет следующий формат: userId, movieId, rating, timestamp 
- <b>«tags.csv»</b> - теги. Каждая строка этого файла после строки заголовка представляет собой один тег, примененный к одному фильму одним пользователем, и имеет следующий формат: userId, movieId, tag, timestamp.Теги — это метаданные о фильмах, созданные пользователем. Каждый тег обычно представляет собой одно слово или короткую фразу. Смысл, значение и назначение конкретного тега определяет каждый пользователь.

In [3]:
movies = pd.read_csv('full_data/movies.csv')
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [4]:
links = pd.read_csv('full_data/links.csv')
links.head(3)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0


In [5]:
movies.shape, links.shape

((86537, 3), (86537, 3))

В полном наборе содержаться более 86 тысяч записей. Следовательно в период с 2017 - 2023 было выпущено более 20000 фильмов.

In [6]:
movies.loc[~movies.movieId.isin(links.movieId)]

,movieId,title,genres


Индикаторы в файлах links и movies полностью совпадают. Но для создания рекомендательной системы мне нужны метаданные, ключевые слова и информация о съемочной группы и актерах, которые отсутствуют у новых фильмов. Решением данного вопроса стал сбор данных с помощью api, предоставленный api.themoviedb.org. 

Для сбора данных потребуется индикатор tmdbId.

In [7]:
links.isna().sum()

movieId      0
imdbId       0
tmdbId     126
dtype: int64

У 126 записей отсутствует индикатор tmdbId. На api.themoviedb.org есть возможность поиска tmdbId через imdbId, чем я и воспользуюсь. Для того, чтобы использовать его, требуется получить API key. Инструкцию как это сделать можно найти на [сайте](https://developer.themoviedb.org/docs). 

Так как я уже получила свой ключ, я импортирую его в ноутбук.

In [2]:
import configparser
config = configparser.ConfigParser()
config.read("config.ini", encoding='utf-8')

token = config["API"]["token"]
api_key = config["API"]["api_key"]

In [12]:
def get_tmdbId(imdbId):
    a = 7 - len(str(imdbId))
    if a > 0:
        imdb_id = 'tt' + '0'*a + str(imdbId)
    else:
        imdb_id = 'tt' + str(imdbId)
    url = f"https://api.themoviedb.org/3/find/{imdb_id}?external_source=imdb_id"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers)
    if not response.ok:
        print(response.text)
        return np.nan
    if not response.json()['movie_results']:
        return np.nan    
    tmdbId = response.json()['movie_results'][0]['id']
    return tmdbId

In [13]:
imdbId_list = links.loc[links.tmdbId.isna(), 'imdbId']

In [14]:
tmdbId_list = imdbId_list.map(get_tmdbId)

In [15]:
links.loc[links.tmdbId.isna(), 'tmdbId'] = tmdbId_list

In [16]:
links.isna().sum()

movieId     0
imdbId      0
tmdbId     58
dtype: int64

Не получилось собрать все tmdbId, но так как отсутствие 58 записей никак не скажется на построении рекамендательной системы, то я удалю пропуски и поменяю формат данных колонки tmdbId на целочисленный.

In [17]:
links = links.drop(links[links.tmdbId.isna()].index)

In [18]:
links.tmdbId = links.tmdbId.astype('int')

In [ ]:
links.to_csv('full_data/links.csv', index=False)

Сначала проверим работоспособность и полученные данные запроса метаданных фильма "Toy Story", который уже у нас есть.

In [23]:
tmdbId = 862
url = f"https://api.themoviedb.org/3/movie/{tmdbId}?append_to_response=production_companies&language=en-US"

headers = {
    "accept": "application/json",
    "Authorization": token
}

response = requests.get(url, headers=headers)

response.json()

{'adult': False,
 'backdrop_path': '/lxD5ak7BOoinRNehOCA85CQ8ubr.jpg',
 'belongs_to_collection': {'id': 10194,
  'name': 'Toy Story Collection',
  'poster_path': '/rki5qLuwb0xnnE9seehxO9TlLhW.jpg',
  'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'},
 'budget': 30000000,
 'genres': [{'id': 16, 'name': 'Animation'},
  {'id': 12, 'name': 'Adventure'},
  {'id': 10751, 'name': 'Family'},
  {'id': 35, 'name': 'Comedy'}],
 'homepage': 'http://toystory.disney.com/toy-story',
 'id': 862,
 'imdb_id': 'tt0114709',
 'original_language': 'en',
 'original_title': 'Toy Story',
 'overview': "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",
 'popularity': 115.249,
 'poster_path': '/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg',
 'production_companies': [{'id'

In [20]:
movies_metadata.loc[movies_metadata.id == '862'][['title', 'vote_average', 'vote_count', 'popularity', 'genres']].T

,0
title,Toy Story
vote_average,7.7
vote_count,5415.0
popularity,21.946943
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."


Те данные, которые у нас есть, и те, которые мы получили, не совпадают. Это не удивительно, так как с прошествием 6 лет этот фильм могло просмотреть большее количество людей, в следствии чего количество голосов, средняя оценка и значение популярности может поменяться. Так же были небольшие корректировки в жанре. 

Делаем вывод, что метаданные подвержены изменениям с течением времени и их нужно обновить.

Что касается информации о ключевых словах и информации об актерах и съемочной группе, то маловероятно, что такая информация будет изменена. Поэтому мы возьмем те данные, которые уже есть.

Создаем датафреймы, в который будем вносить данные:

In [21]:
movies = pd.DataFrame({'tmdbId': links.tmdbId.tolist()})

In [24]:
detail_keys = response.json().keys()

In [25]:
for d_key in detail_keys:
    movies.insert(movies.shape[1], d_key, np.nan)
movies.head(2)

,tmdbId,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Заполняем credits и keywords данными:

In [26]:
credits = pd.DataFrame({'tmdbId': links.tmdbId.tolist()})
keywords = pd.DataFrame({'tmdbId': links.tmdbId.tolist()})

old_keywords.rename(columns={'id': 'tmdbId'}, inplace=True)
keywords = keywords.merge(old_keywords, on='tmdbId', how='left')

old_credits.rename(columns={'id': 'tmdbId'}, inplace=True)
credits = credits.merge(old_credits, on='tmdbId', how='left')

credits_keys = ['cast', 'crew']
keywords_key = ['keywords']

In [79]:
import time
import datetime

def get_info(tmdbId):
    try:
        headers = {
            "accept": "application/json",
            "Authorization": token
            }
        
        # get details
        # часто нет tagline и homepage, поэтому общая сумма пропусков по колонкам не должна превышать 2 
        if movies.loc[movies.tmdbId == tmdbId].isna().sum().sum() > 2:
            url = f"https://api.themoviedb.org/3/movie/{tmdbId}??api_key={api_key}&append_to_response=production_companies&language=en-US"

            response = requests.get(url, headers=headers)
            if not response.ok:
                print(f'{datetime.datetime.now().strftime("%H:%M:%S")} -  Не получили details данные {tmdbId}')
                # сохраняем индификаторы записей, которые не нашли, чтобы в случае повторного запуска кода,
                # не проходить по ним заново
                with open('error_requests.txt', 'a') as f:
                    f.write(str(tmdbId) + '\n')
                    print("/tСохранен в error_requests.txt ")
            else:
                details = dict(response.json())
                for d_key in detail_keys:
                    if d_key in details.keys():
                        movies.loc[movies.tmdbId == tmdbId, d_key] = str(details[d_key])

        # get keywords
        # было замечено, что в датасете вместо nan записан [], который приравнивается к пропуску.
        if keywords.loc[keywords.tmdbId == tmdbId].isna().sum().sum() > 0 or keywords.loc[keywords.tmdbId == tmdbId, 'keywords'].values[0] == '[]':
            url = f"https://api.themoviedb.org/3/movie/{tmdbId}/keywords?api_key={api_key}"
            response = requests.get(url, headers=headers)
            if not response.ok:
                print(f'{ datetime.datetime.now().strftime("%H:%M:%S")} - Не получили keywords данные {tmdbId}')
            else:
                keywords_set = dict(response.json())
                for k_key in keywords_key:
                    if k_key in keywords_set.keys():
                        keywords.loc[keywords.tmdbId == tmdbId, k_key] = str(keywords_set[k_key])

        # get credits
        if credits.loc[credits.tmdbId == tmdbId].isna().sum().sum() > 0 or credits.loc[credits.tmdbId == tmdbId, 'cast'].values[0] == '[]' or credits.loc[credits.tmdbId == tmdbId, 'crew'].values[0] == '[]':
            url = f"https://api.themoviedb.org/3/movie/{tmdbId}/credits?api_key={api_key}&language=en-US"
            response = requests.get(url, headers=headers)
            if not response.ok:
                print(f'{ datetime.datetime.now().strftime("%H:%M:%S")} - Не получили credits данные {tmdbId}')
            else:
                credits_set = dict(response.json())
                for c_key in credits_keys:
                    if c_key in credits_set.keys():
                        credits.loc[credits.tmdbId == tmdbId, c_key] = str(credits_set[c_key])
        
        # save files
        if tmdbId%5 == 0:
            movies.to_csv('full_data/movie_metadata.csv', index=False)
            keywords.to_csv('full_data/keywords.csv', index=False)
            credits.to_csv('full_data/credits.csv', index=False)
            print(f'{ datetime.datetime.now().strftime("%H:%M:%S")} - Сохранение датасетов. Сейчас есть {movies.adult.isna().sum()} пропусков')
            
    except Exception as e:
        print(e)
        time.sleep(120)
        print(f'Sleep at {tmdbId}')

In [139]:
tmdbIds = movies.loc[(movies.adult.isna()) & (~movies.tmdbId.isin(error_tmdbIds)), 'tmdbId']

In [140]:
tmdbIds = tmdbIds.drop_duplicates()

Запускаем функцию сбора данных

In [141]:
tmdbIds.map(get_info)

19:53:35 -  Не получили details данные 538286
/tСохранен в error_requests.txt 
19:53:37 - Не получили keywords данные 538286
19:53:38 - Не получили credits данные 538286
19:53:39 -  Не получили details данные 12773
/tСохранен в error_requests.txt 
19:53:41 - Не получили keywords данные 12773
19:53:43 - Не получили credits данные 12773
19:53:44 -  Не получили details данные 17882
/tСохранен в error_requests.txt 
19:53:46 - Не получили keywords данные 17882
19:53:48 - Не получили credits данные 17882
19:53:49 -  Не получили details данные 68149
/tСохранен в error_requests.txt 
19:53:50 - Не получили keywords данные 68149
19:53:51 - Не получили credits данные 68149
19:53:53 -  Не получили details данные 14980
/tСохранен в error_requests.txt 
19:54:10 - Сохранение датасетов. Сейчас есть 5063 пропусков
19:54:11 -  Не получили details данные 164721
/tСохранен в error_requests.txt 
19:54:13 - Не получили keywords данные 164721
19:54:14 - Не получили credits данные 164721
19:54:15 -  Не получи

19:59:31 -  Не получили details данные 202241
/tСохранен в error_requests.txt 
19:59:32 -  Не получили details данные 36663
/tСохранен в error_requests.txt 
19:59:33 - Не получили keywords данные 36663
19:59:34 - Не получили credits данные 36663
19:59:35 -  Не получили details данные 71982
/tСохранен в error_requests.txt 
19:59:37 -  Не получили details данные 67636
/tСохранен в error_requests.txt 
19:59:39 - Не получили keywords данные 67636
19:59:40 - Не получили credits данные 67636
19:59:41 -  Не получили details данные 51765
/tСохранен в error_requests.txt 
19:59:42 - Не получили credits данные 51765
19:59:54 - Сохранение датасетов. Сейчас есть 5063 пропусков
19:59:56 -  Не получили details данные 75375
/tСохранен в error_requests.txt 
19:59:57 - Не получили keywords данные 75375
19:59:58 - Не получили credits данные 75375
20:00:10 - Сохранение датасетов. Сейчас есть 5063 пропусков
20:00:11 -  Не получили details данные 58423
/tСохранен в error_requests.txt 
20:00:12 - Не получили

20:04:42 -  Не получили details данные 252993
/tСохранен в error_requests.txt 
20:04:43 - Не получили keywords данные 252993
20:04:44 - Не получили credits данные 252993
20:04:46 -  Не получили details данные 239471
/tСохранен в error_requests.txt 
20:04:47 - Не получили keywords данные 239471
20:04:48 - Не получили credits данные 239471
20:04:49 -  Не получили details данные 245170
/tСохранен в error_requests.txt 
20:04:51 - Не получили credits данные 245170
20:05:03 - Сохранение датасетов. Сейчас есть 5063 пропусков
20:05:04 -  Не получили details данные 110414
/tСохранен в error_requests.txt 
20:05:06 - Не получили keywords данные 110414
20:05:08 - Не получили credits данные 110414
20:05:10 -  Не получили details данные 313289
/tСохранен в error_requests.txt 
20:05:11 - Не получили keywords данные 313289
20:05:14 - Не получили credits данные 313289
20:05:16 -  Не получили details данные 370353
/tСохранен в error_requests.txt 
20:05:17 -  Не получили details данные 218277
/tСохранен 

20:09:45 -  Не получили details данные 197592
/tСохранен в error_requests.txt 
20:09:46 -  Не получили details данные 25093
/tСохранен в error_requests.txt 
20:09:47 - Не получили keywords данные 25093
20:09:48 - Не получили credits данные 25093
20:09:50 -  Не получили details данные 13535
/tСохранен в error_requests.txt 
20:10:01 - Сохранение датасетов. Сейчас есть 5063 пропусков
20:10:03 -  Не получили details данные 57622
/tСохранен в error_requests.txt 
20:10:04 - Не получили keywords данные 57622
20:10:05 - Не получили credits данные 57622
20:10:06 -  Не получили details данные 162864
/tСохранен в error_requests.txt 
20:10:08 -  Не получили details данные 67456
/tСохранен в error_requests.txt 
20:10:09 - Не получили keywords данные 67456
20:10:10 - Не получили credits данные 67456
20:10:11 -  Не получили details данные 53285
/tСохранен в error_requests.txt 
20:10:12 - Не получили keywords данные 53285
20:10:14 - Не получили credits данные 53285
20:10:26 - Сохранение датасетов. Сей

20:14:33 - Не получили keywords данные 141334
20:14:35 - Не получили credits данные 141334
20:14:36 -  Не получили details данные 108497
/tСохранен в error_requests.txt 
20:14:39 - Не получили keywords данные 108497
20:14:40 - Не получили credits данные 108497
20:14:42 -  Не получили details данные 114691
/tСохранен в error_requests.txt 
20:14:44 - Не получили keywords данные 114691
20:14:45 - Не получили credits данные 114691
20:14:47 -  Не получили details данные 99022
/tСохранен в error_requests.txt 
20:14:48 - Не получили keywords данные 99022
20:14:50 -  Не получили details данные 353653
/tСохранен в error_requests.txt 
20:14:51 - Не получили keywords данные 353653
20:14:52 -  Не получили details данные 25180
/tСохранен в error_requests.txt 
20:14:53 - Не получили keywords данные 25180
20:15:05 - Сохранение датасетов. Сейчас есть 5063 пропусков
20:15:06 -  Не получили details данные 278604
/tСохранен в error_requests.txt 
20:15:07 -  Не получили details данные 246699
/tСохранен в 

...

02:57:02 - Сохранение датасетов. Сейчас есть 1856 пропусков
02:57:19 - Сохранение датасетов. Сейчас есть 1855 пропусков
02:57:48 - Сохранение датасетов. Сейчас есть 1850 пропусков
02:58:06 - Сохранение датасетов. Сейчас есть 1848 пропусков
02:58:21 - Сохранение датасетов. Сейчас есть 1847 пропусков
02:58:36 - Сохранение датасетов. Сейчас есть 1846 пропусков
02:59:01 - Сохранение датасетов. Сейчас есть 1842 пропусков
02:59:17 - Сохранение датасетов. Сейчас есть 1841 пропусков
02:59:53 - Сохранение датасетов. Сейчас есть 1836 пропусков
03:00:20 - Сохранение датасетов. Сейчас есть 1832 пропусков
03:00:36 - Сохранение датасетов. Сейчас есть 1831 пропусков
03:00:56 - Сохранение датасетов. Сейчас есть 1829 пропусков
03:01:17 - Сохранение датасетов. Сейчас есть 1826 пропусков
03:01:45 - Сохранение датасетов. Сейчас есть 1821 пропусков
03:02:37 - Сохранение датасетов. Сейчас есть 1811 пропусков
03:02:53 - Сохранение датасетов. Сейчас есть 1810 пропусков
03:03:26 - Сохранение датасетов. Сейчас 

03:59:16 - Сохранение датасетов. Сейчас есть 1275 пропусков
03:59:32 - Сохранение датасетов. Сейчас есть 1274 пропусков
03:59:57 - Сохранение датасетов. Сейчас есть 1270 пропусков
04:01:08 - Сохранение датасетов. Сейчас есть 1257 пропусков
04:01:13 -  Не получили details данные 101999
/tСохранен в error_requests.txt 
04:01:14 - Не получили keywords данные 101999
04:01:15 - Не получили credits данные 101999
04:01:53 - Сохранение датасетов. Сейчас есть 1251 пропусков
04:02:22 - Сохранение датасетов. Сейчас есть 1246 пропусков
04:03:01 -  Не получили details данные 540387
/tСохранен в error_requests.txt 
04:03:02 - Не получили keywords данные 540387
04:03:03 - Не получили credits данные 540387
04:03:29 - Сохранение датасетов. Сейчас есть 1230 пропусков
04:03:46 - Сохранение датасетов. Сейчас есть 1229 пропусков
04:04:09 - Сохранение датасетов. Сейчас есть 1227 пропусков
04:04:39 - Сохранение датасетов. Сейчас есть 1222 пропусков
04:05:13 -  Не получили details данные 548004
/tСохранен в e

04:30:55 - Не получили credits данные 62444
04:30:56 -  Не получили details данные 53986
/tСохранен в error_requests.txt 
04:30:57 - Не получили keywords данные 53986
04:30:58 - Не получили credits данные 53986
04:30:59 -  Не получили details данные 384975
/tСохранен в error_requests.txt 
04:31:00 - Не получили keywords данные 384975
04:31:02 - Не получили credits данные 384975
04:31:13 - Сохранение датасетов. Сейчас есть 981 пропусков
04:31:14 -  Не получили details данные 465803
/tСохранен в error_requests.txt 
04:31:15 - Не получили keywords данные 465803
04:31:16 - Не получили credits данные 465803
04:31:17 -  Не получили details данные 467080
/tСохранен в error_requests.txt 
04:31:19 - Не получили keywords данные 467080
04:31:20 - Не получили credits данные 467080
04:31:32 - Сохранение датасетов. Сейчас есть 981 пропусков
04:31:35 -  Не получили details данные 578715
/tСохранен в error_requests.txt 
04:31:36 - Не получили keywords данные 578715
04:31:37 - Не получили credits данны

04:36:37 -  Не получили details данные 360086
/tСохранен в error_requests.txt 
04:36:39 - Не получили keywords данные 360086
04:36:40 - Не получили credits данные 360086
04:36:43 -  Не получили details данные 92246
/tСохранен в error_requests.txt 
04:36:44 - Не получили keywords данные 92246
04:36:45 - Не получили credits данные 92246
04:36:47 -  Не получили details данные 210590
/tСохранен в error_requests.txt 
04:36:49 - Не получили keywords данные 210590
04:36:51 - Не получили credits данные 210590
04:37:03 - Сохранение датасетов. Сейчас есть 980 пропусков
04:37:04 -  Не получили details данные 629861
/tСохранен в error_requests.txt 
04:37:05 - Не получили keywords данные 629861
04:37:06 - Не получили credits данные 629861
04:37:07 -  Не получили details данные 636391
/tСохранен в error_requests.txt 
04:37:08 - Не получили keywords данные 636391
04:37:09 - Не получили credits данные 636391
04:37:10 -  Не получили details данные 224302
/tСохранен в error_requests.txt 
04:37:11 - Не п

596      None
4092     None
4452     None
4952     None
7381     None
         ... 
64432    None
64433    None
64557    None
64713    None
64720    None
Name: tmdbId, Length: 4869, dtype: object

In [137]:
error_tmdbId = []
with open('error_requests.txt', 'r') as f:
    for line in f.readlines():
        error_tmdbId.append(int(line.strip()))

In [142]:
movies.isna().sum()

tmdbId                       0
adult                      980
backdrop_path              980
belongs_to_collection      980
budget                     980
genres                     980
homepage                 10229
id                         980
imdb_id                    986
original_language          980
original_title             980
overview                  1140
popularity                 980
poster_path                980
production_companies       980
production_countries       980
release_date              1005
revenue                    980
runtime                    980
spoken_languages           980
status                     980
tagline                   9548
title                      980
video                      980
vote_average               980
vote_count                 980
dtype: int64

Не получилось собрать 980 записей. Удалим их из датасетов

In [184]:
tmdbId_empty_records = movies.loc[movies.adult.isna(), 'tmdbId'].tolist()

In [213]:
empty_records_index = movies.loc[movies.adult.isna()].index

In [217]:
movies = movies.drop(empty_records_index)
movies.drop_duplicates(inplace=True)

In [271]:
keywords = keywords.loc[keywords.tmdbId.isin(movies.tmdbId.tolist())].drop_duplicates()

In [273]:
credits = credits.loc[credits.tmdbId.isin(movies.tmdbId.tolist())].drop_duplicates()

In [298]:
movies.shape, keywords.shape, credits.shape

((85463, 26), (85463, 2), (85469, 3))

У credits есть лишние 6 записей, которые говорят о том, что существуют дублирующие индификаторы с разной информацией. Найдем их и вывидем на экран 

In [292]:
tmdbId_with_two_records = [159849, 132641, 110428, 4912, 298721, 99080]
credits.loc[credits.tmdbId.isin(tmdbId_with_two_records)]

,tmdbId,cast,crew
844,132641,"[{'cast_id': 4, 'character': 'Mihoko Nakagawa'...","[{'credit_id': '52fe4b9ac3a368484e190d25', 'de..."
845,132641,"[{'cast_id': 4, 'character': 'Mihoko Nakagawa'...","[{'credit_id': '56365ed9925141285701b06e', 'de..."
5887,4912,"[{'cast_id': 15, 'character': 'Chuck Barris', ...","[{'credit_id': '52fe43e2c3a36847f80760b5', 'de..."
5888,4912,"[{'cast_id': 15, 'character': 'Chuck Barris', ...","[{'credit_id': '52fe43e2c3a36847f80760a9', 'de..."
21726,110428,"[{'cast_id': 3, 'character': 'Camille Claudel'...","[{'credit_id': '52fe4ad6c3a36847f81e461b', 'de..."
21727,110428,"[{'cast_id': 3, 'character': 'Camille Claudel'...","[{'credit_id': '577ed5389251416976004432', 'de..."
33560,298721,"[{'cast_id': 1, 'character': 'Jenjira', 'credi...","[{'credit_id': '5448c8efc3a3680fb4001582', 'de..."
33561,298721,"[{'cast_id': 1, 'character': 'Jenjira', 'credi...","[{'credit_id': '5739ceeac3a3683c9d000c12', 'de..."
36894,159849,"[{'cast_id': 12, 'character': 'Narrator', 'cre...","[{'credit_id': '52fe4c229251416c910f10d5', 'de..."
36895,159849,"[{'cast_id': 12, 'character': 'Narrator', 'cre...","[{'credit_id': '52fe4c229251416c910f10d5', 'de..."


Так как не понятно, какая из строчек является истинной, то заново найдем значения для них.

In [300]:
credits = credits.drop([845, 5888, 21727, 33561, 36895, 40715])

In [309]:
for tmdbId in tmdbId_with_two_records:
    url = f"https://api.themoviedb.org/3/movie/{tmdbId}/credits?api_key={api_key}&language=en-US"
    headers = {
                "accept": "application/json",
                "Authorization": token
                }
    response = requests.get(url, headers=headers)
    if not response.ok:
        print(f'{ datetime.datetime.now().strftime("%H:%M:%S")} - Не получили credits данные {tmdbId}')
    else:
        credits_set = dict(response.json())
        for c_key in credits_keys:
            if c_key in credits_set.keys():
                credits.loc[credits.tmdbId == tmdbId, c_key] = str(credits_set[c_key])
        print(tmdbId)

ProxyError: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Max retries exceeded with url: /3/movie/159849/credits?api_key=c4cb6ffa32ab677f475d001b4cbd14a7&language=en-US (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 403 Forbidden')))

In [301]:
movies.shape, keywords.shape, credits.shape

((85463, 26), (85463, 2), (85463, 3))

Наши файлы готовы. Всего в датасете собрана информация о 85463 фильмов. Сохраняем конечный результат

In [303]:
movies.to_csv('full_data/movie_metadata_final.csv', index=False)
keywords.to_csv('full_data/keywords_final.csv', index=False)
credits.to_csv('full_data/credits_final.csv', index=False)